In [ ]:
from itertools import product
from collections import defaultdict

class Vertex:
    """Class representing a vertex in the q-ary graph."""
    def __init__(self, sequence):
        self.sequence = sequence  # Store the sequence (e.g., '001')
        self.Bx = self.generate_Bx()  # Store the B(x) set for the vertex

    def generate_Bx(self):
        """Generate the set B(x) by removing each position from the sequence."""
        Bx = set()
        for i in range(len(self.sequence)):
            Bx.add(self.sequence[:i] + self.sequence[i + 1:])
        return Bx

    def __str__(self):
        return f"Vertex({self.sequence})"

    def __repr__(self):
        return self.__str__()

class QAryGraph:
    """Graph of q-ary sequences with length n as vertices."""
    def __init__(self, q, n):
        self.q = q  # q- ary sequences
        self.n = n  # Length of the sequences
        self.vertices = self.generate_vertices()  # List of Vertex objects
        self.adj_list = defaultdict(list)  # Adjacency list
        self.adj_matrix = [[0] * len(self.vertices) for _ in range(len(self.vertices))]
        self.vertex_indices = {v.sequence: i for i, v in enumerate(self.vertices)}  # Map sequence to index

    def generate_vertices(self):
        """Generate all q-ary sequences of length n as Vertex objects."""
        sequences = [''.join(map(str, seq)) for seq in product(range(self.q), repeat=self.n)]
        return [Vertex(sequence) for sequence in sequences]

    def are_connected(self, v1, v2):
        """Check if two vertices are connected (B(x) ∩ B(y) = ∅)."""
        return v1.Bx.isdisjoint(v2.Bx)

    def build_graph(self):
        """Build the graph by constructing the adjacency list and matrix."""
        for i, v1 in enumerate(self.vertices):
            for j, v2 in enumerate(self.vertices):
                if i < j and self.are_connected(v1, v2):
                    # Add to adjacency list
                    self.adj_list[v1.sequence].append(v2.sequence)
                    self.adj_list[v2.sequence].append(v1.sequence)
                    # Update adjacency matrix
                    self.adj_matrix[i][j] = 1
                    self.adj_matrix[j][i] = 1

    def get_adjacency_list(self):
        """Return the adjacency list."""
        return dict(self.adj_list)

    def get_adjacency_matrix(self):
        """Return the adjacency matrix."""
        return self.adj_matrix

    def bron_kerbosch(self, R, P, X, cliques):
        """Bron-Kerbosch algorithm to find all maximal cliques."""
        if not P and not X:
            cliques.append(R)
            return
        for v in list(P):
            neighbors = set(self.adj_list[v])
            self.bron_kerbosch(R | {v}, P & neighbors, X & neighbors, cliques)
            P.remove(v)
            X.add(v)

    def max_clique(self):
        """Find all maximum cliques and their size."""
        cliques = []
        P = set(v.sequence for v in self.vertices)
        self.bron_kerbosch(set(), P, set(), cliques)

        # Find the size of the maximum cliques
        max_size = max(len(clique) for clique in cliques)

        # Collect all cliques with the maximum size
        max_cliques = [clique for clique in cliques if len(clique) == max_size]

        return max_size, max_cliques

# Example usage
q = 2  # Binary sequences {0,1}
n = 7  # Length of each sequence is 3
graph = QAryGraph(q, n)
graph.build_graph()

# print("Adjacency List:")
# print(graph.get_adjacency_list())

# print("\nAdjacency Matrix:")
# for row in graph.get_adjacency_matrix():
#     print(row)

print("\nMax Clique:")
max_size, max_cliques = graph.max_clique()
print(f"Size of the maximum clique: {max_size}")
print(f"All max cliques: {max_cliques}")



Max Clique:
